In [ ]:
!pip install torch
!pip install torchaudio
!pip install torch diffusers
!pip install torchvision
!pip install transformers
!pip install accelerate
!pip install translate
!pip install rutube-downloader m3u8 alive_progress
!pip install opencv-python
!pip install beautifulsoup4
!pip install detectron2

In [ ]:
!python -m pip install pyyaml
import sys, os, distutils.core
!git clone "https://github.com/facebookresearch/detectron2"
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i "s/OPENCV=0/OPENCV=1/" Makefile
!sed -i "s/GPU=0/GPU=1/" Makefile
!sed -i "s/CUDNN=0/CUDNN=1/" Makefile
!sed -i "s/CUDNN_HALF=0/CUDNN_HALF=1/" Makefile
!make

In [ ]:
import cv2
from typing import Tuple
import numpy as np
import os.path
import pandas as pd
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
from translate import Translator
from flask import Flask, render_template, request, redirect, jsonify
import requests
from rutube import Rutube
from bs4 import BeautifulSoup
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import os, json, random
#from google.colab.patches import cv2_imshow
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

In [ ]:
nums = ['person\n', 'bicycle\n', 'car\n', 'motorbike\n', 'aeroplane\n', 'bus\n', 'train\n', 'truck\n', 'boat\n', 'traffic light\n', 'fire hydrant\n', 'stop sign\n', 'parking meter\n', 'bench\n', 'bird\n', 'cat\n', 'dog\n', 'horse\n', 'sheep\n', 'cow\n', 'elephant\n', 'bear\n', 'zebra\n', 'giraffe\n', 'backpack\n', 'umbrella\n', 'handbag\n', 'tie\n', 'suitcase\n', 'frisbee\n', 'skis\n', 'snowboard\n', 'sports ball\n', 'kite\n', 'baseball bat\n', 'baseball glove\n', 'skateboard\n', 'surfboard\n', 'tennis racket\n', 'bottle\n', 'wine glass\n', 'cup\n', 'fork\n', 'knife\n', 'spoon\n', 'bowl\n', 'banana\n', 'apple\n', 'sandwich\n', 'orange\n', 'broccoli\n', 'carrot\n', 'hot dog\n', 'pizza\n', 'donut\n', 'cake\n', 'chair\n', 'sofa\n', 'pottedplant\n', 'bed\n', 'diningtable\n', 'toilet\n', 'tvmonitor\n', 'laptop\n', 'mouse\n', 'remote\n', 'keyboard\n', 'cell phone\n', 'microwave\n', 'oven\n', 'toaster\n', 'sink\n', 'refrigerator\n', 'book\n', 'clock\n', 'vase\n', 'scissors\n', 'teddy bear\n', 'hair drier\n', 'toothbrush\n']

In [ ]:
model_id = ''
prompt = ''
path = ''
description = ''
title = ''

In [ ]:
name = title
file_name = path
description = description

In [ ]:
import os
from random import randint
import shutil


def split_video(video_path):
    file_name = video_path.rstrip('.mp4')
    os.mkdir('/content/Frames')
    video = cv2.VideoCapture(f'/content/{video_path}')
    frame_count = 0
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = total_frames // 60
    count = 0
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        if frame_count % 60 == 0:
            cv2.waitKey(1)
            cv2.imwrite(fr"/content/Frames/{count}.jpg", frame)
            count += 1
        frame_count += 1
    video.release()
    cv2.destroyAllWindows()
    return frame_interval
    # shutil.rmtree('c:\sochi\Frames')

from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
translator = Translator(from_lang='ru', to_lang='en')

In [ ]:
inp = input('1. У меня есть файл видео\n2. У меня есть ссылка на видео\n3. Поиск для автора\n')

if inp == '1':
  file_name = input('Введите путь до файла: ')
  description = input('Введите описание видео: ')
  wish = input('Введите свои пожелания: ')
  text = input('Введите желаемый текст для картинки: ')
  models = {
            'anime': 'hakurei/waifu-diffusion',
            '3d': 'runwayml/stable-diffusion-v1-5',
            'realistic': 'CompVis/stable-diffusion-v1-4'
  }
  model_id = models[input('Введите стиль картинки: realistic, anime, 3d\n')]
  prompt = wish
  pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to('cuda')
  frame_interval = split_video(file_name)

  !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
  counts_dict = {}
  from google.colab.patches import cv2_imshow
  path = '/content/Frames/'
  for i in range(frame_interval):
      im = cv2.imread(f'{path}{i}.jpg')
      cfg = get_cfg()
      # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
      cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
      cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
      # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
      cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
      predictor = DefaultPredictor(cfg)
      outputs = predictor(im)
      v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
      out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      for i in list(outputs["instances"].pred_classes):
          if f'{nums[i]}' not in counts_dict.keys():
            counts_dict[f'{nums[i]}'] = 0
          counts_dict[f'{nums[i]}'] += 1
      cv2_imshow(out.get_image()[:, :, ::-1])
  shutil.rmtree('/content/Frames')
  sorted_dict = [i[0] for i in sorted(counts_dict.items()) if i[1] > 5][:5]
  prompt += ' '.join(sorted_dict)
  with autocast("cuda"):
    image = pipe(prompt, guidance_scale=6)["images"][0]
    image.save(f"/content/image.png")
    img = Image.open('/content/image.png')
    draw = ImageDraw.Draw(img)
    x, y = img.size
    font = ImageFont.truetype('/content/News.ttf', choose_font_size(x, y))
    width = font.getlength(text)
    pos = (x - width) / 2
    draw.text((pos, y * 0.75), translator.translate(text), (255, 255, 255), font=font)
    img.save(f'/content/sample-out.jpg')
    cv2_imshow(cv2.imread(f'/content/sample-out.jpg'))
elif inp == '2':
  link = input('Введите ссылку на видео: ')
  rt = Rutube(link)
  rt.playlist[-1].download()
  html = requests.get(link).text
  soup = BeautifulSoup(html, 'lxml')
  title = soup.find(class_='video-pageinfo-container-module__videoTitle').next.next.next
  file_name = f'{title}.mp4'
  models = {
            'anime': 'hakurei/waifu-diffusion',
            '3d': 'runwayml/stable-diffusion-v1-5',
            'realistic': 'CompVis/stable-diffusion-v1-4'
  }
  model_id = models[input('Введите стиль картинки: realistic, anime, 3d\n')]
  text = input('Введите желаемый текст для картинки: ')
  wish = input('Введите пожелания к видео: ')
  description = input('Введите описание к видео: ')
  category = html[html.find("category_url") + 12:]
  category = category[:category.find("}")]
  category = category[category.find("name")+5:]
  category = category[category.find("\"")+1:category.rfind("\"")]
  prompt = ' '.join([wish, category, title])
  file_name = f'/content/{i}.mp4'
  prompt = ' '.join([wish, description])
  prompt = translator.translate(prompt)
  pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to('cuda')
  frame_interval = split_video(file_name)

  !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
  counts_dict = {}
  from google.colab.patches import cv2_imshow
  path = '/content/Frames/'
  for i in range(frame_interval):
      im = cv2.imread(f'{path}{i}.jpg')
      cfg = get_cfg()
      # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
      cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
      cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
      # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
      cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
      predictor = DefaultPredictor(cfg)
      outputs = predictor(im)
      v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
      out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      for i in list(outputs["instances"].pred_classes):
          if f'{nums[i]}' not in counts_dict.keys():
            counts_dict[f'{nums[i]}'] = 0
          counts_dict[f'{nums[i]}'] += 1
      cv2_imshow(out.get_image()[:, :, ::-1])
  shutil.rmtree('/content/Frames')
  sorted_dict = [i[0] for i in sorted(counts_dict.items()) if i[1] > 5][:5]
  prompt += ' '.join(sorted_dict)
  with autocast("cuda"):
    image = pipe(prompt, guidance_scale=6)["images"][0]
    image.save(f"/content/image.png")
    img = Image.open('/content/image.png')
    draw = ImageDraw.Draw(img)
    x, y = img.size
    font = ImageFont.truetype('/content/News.ttf', choose_font_size(x, y))
    width = font.getlength(text)
    pos = (x - width) / 2
    draw.text((pos, y * 0.75), translator.translate(text), (255, 255, 255), font=font)
    img.save(f'/content/sample-out.jpg')
    cv2_imshow(cv2.imread(f'/content/sample-out.jpg'))
else:
  dicts = []
  author = input('Введите имя автора: ')
  wish = input('Пожелания: ')
  theme = input('Тематика канала: ')
  text = input('Желаемый текст к картинке: ')
  models = {
            'anime': 'hakurei/waifu-diffusion',
            '3d': 'runwayml/stable-diffusion-v1-5',
            'realistic': 'CompVis/stable-diffusion-v1-4'
  }
  model_id = models[input('Введите стиль картинки: realistic, anime, 3d\n')]
  prompt = ' '.join([wish, theme])
  prompt = translator.translate(prompt)
  d = dict()
  for i in range(0, data.shape[0]):
    try:
      d[data.channel_title[i]].append(i)
    except:
      d[data.channel_title[i]] = [i]
  author_ids = d[author]
  for id in author_ids:
    file_name = f'/content/{id}.mp4'
    prompt = ' '.join([wish, description])
    prompt = translator.translate(prompt)
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to('cuda')
    frame_interval = split_video(file_name)

    !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
    counts_dict = {}
    from google.colab.patches import cv2_imshow
    path = '/content/Frames/'
    for i in range(frame_interval):
        im = cv2.imread(f'{path}{i}.jpg')
        cfg = get_cfg()
        # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
        cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
        # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
        cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
        predictor = DefaultPredictor(cfg)
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        for i in list(outputs["instances"].pred_classes):
            if f'{nums[i]}' not in counts_dict.keys():
              counts_dict[f'{nums[i]}'] = 0
            counts_dict[f'{nums[i]}'] += 1
        cv2_imshow(out.get_image()[:, :, ::-1])
    shutil.rmtree('/content/Frames')
    dicts.append(counts_dict)

  for num_dict in range(1, len(dicts)):
    for cnts in dicts[num_dict]:
        if cnts not in dicts[0]:
            dicts[0][cnts] = dicts[num_dict][cnts]
        dicts[0][cnts] += 1
  sorted_dict = [i[0] for i in sorted(dicts[0].items()) if i[1] > 15][:5]
  prompt = ' '.join(sorted_dict) + ' ' + theme
  pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to('cuda')
  with autocast("cuda"):
    image = pipe(prompt, guidance_scale=6)["images"][0]
    image.save(f"/content/image.png")
    img = Image.open('/content/image.png')
    draw = ImageDraw.Draw(img)
    x, y = img.size
    font = ImageFont.truetype('/content/News.ttf', choose_font_size(x, y))
    width = font.getlength(text)
    pos = (x - width) / 2
    draw.text((pos, y * 0.75), translator.translate(text), (255, 255, 255), font=font)
    img.save(f'/content/sample-out.jpg')
    cv2_imshow(cv2.imread(f'/content/sample-out.jpg'))